In [ ]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import itertools
import pickle
import numpy as np
from numpy import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib
from typing import TypeVar, Dict

import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer

Tensor = TypeVar('torch.tensor')

#matplotlib.use('Agg')

import click
from argparse import Namespace
import ast
import os

import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from typing import TypeVar, Tuple
import gdown
import copy

In [ ]:
#@title Define Hyperparameters

# class_names = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
#                 "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
#                 "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
#                 "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
#                 "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
#                 "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
#                 "Wearing_Necktie", "Young"]
# num_classes = 40
# data_shape = [3, 64, 64]
# n_bits = 5
# temp = 0.7
# data_mean = [0.485, 0.456, 0.406]
# data_std = [0.229, 0.224, 0.225]

input_size = 28*28*1 # img_size = (28,28) ---> 28*28=784 in total
batch_size = 200 # the size of input data took for one iteration

In [ ]:
# from torchvision.transforms import v2
# transform_RandomErasing=transforms.Compose([v2.RandomErasing(),
#                               transforms.ToTensor()])
#from torchvision.transforms import v2
#transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=0,std=1.0)])
transform=transforms.Compose([transforms.ToTensor()])
train_data = dsets.FashionMNIST(root = './data', train=True, transform = transform, download = True)
test_data = dsets.FashionMNIST(root = './data', train=False, transform = transform, download = True)

In [ ]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)



In [ ]:
#cvmx=torch.zeros([3*64*64,3*64*64],device=device)
images_list=[]
labels_list=[]
no_batches=len(train_gen)
#images_mean=torch.zeros(3,64,64,device=device)
for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    # images_mean=images_mean+images.mean(0)
    # im=torch.reshape(images,[images.shape[0],3*64*64])
    # cvmx+=torch.matmul(torch.transpose(im,0,1),im)
    if(i<(len(train_gen))):
        images_list.append(images)
        labels_list.append(labels)



test_images_list=[]
test_labels_list=[]
test_no_batches=len(test_gen)
for i ,(images,labels) in enumerate(test_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    if(i<(len(test_gen))):
        test_images_list.append(images)
        test_labels_list.append(labels)

train_data_len=len(train_data)
test_data_len=len(test_data)



In [ ]:
#@title Define model class
import torch.nn as nn
import torch
import torch.nn.functional as F


from typing import TypeVar, Tuple

Tensor = TypeVar('torch.tensor')


class NeuralNet(torch.nn.Module):
    """
    base class for all NN classifiers
    """
    def __init__(self):
        super().__init__()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.BatchNorm2d):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.Linear):
                torch.nn.init.normal_(m.weight, 0, 0.01)
                torch.nn.init.constant_(m.bias, 0)

class multinomial(NeuralNet):
    """
    CNN for (binary) classification for CelebA, CheXpert
    """

    def __init__(self,
                 num_classes: int = 10,
                 flattened_size: int = 28*28):
        """Builder."""
        super(multinomial, self).__init__()

        self.last_layer=nn.Sequential(
            nn.Flatten(),
            nn.Linear(flattened_size, num_classes)
        )

    def forward(self, x: Tensor) -> Tensor:
        """Perform forward."""

        x=self.last_layer(x)

        return x

    def classify(self, x: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
        net_out = self.forward(x)
        acc = F.softmax(net_out, dim=1)
        class_idx = torch.max(net_out, 1)[1]

        return acc, acc[0, class_idx], class_idx
    




In [ ]:
# net=Fashion_MNIST_CNN()
# n=0
# for par in net.parameters():
#     n+=par.numel()

# n

In [ ]:
#@title Define loss-function & optimizer
loss_function = nn.CrossEntropyLoss()


def images_regulariser(net): 
    li_reg_loss = 0
    penalized     = [p for name,p in net.named_parameters() if 'bias' not in name]
    not_penalized = [p for name,p in net.named_parameters() if 'bias' in name]
    for p in penalized:
        li_reg_loss += (p**2).sum()*0.5
    #for p in net.parameters():
#        li_reg_loss += (p**2).sum()*0.5
    reg=li_reg_loss/(train_data_len)*l2regconst
    return(reg)


In [ ]:
def addnet(net,net2):
    for param1, param2 in zip(net.parameters(), net2.parameters()):
     param1.data += param2.data

def multiplynet(net,a):
   for param1 in net.parameters():
     param1.data *=a

In [ ]:
from dataclasses import dataclass
@dataclass
class hclass:
    h: Tensor
    eta: Tensor
    etam1g: Tensor
    c11: Tensor
    c21: Tensor
    c22: Tensor

@dataclass
class BAOABhclass:
    h: Tensor
    eta: Tensor
    xc1: Tensor
    xc2: Tensor
    xc3: Tensor
    vc1: Tensor
    vc2: Tensor
    vc3: Tensor


def hper2const(h,gam):
    gh=gam.double()*h.double()
    s=torch.sqrt(4*torch.expm1(-gh/2)-torch.expm1(-gh)+gh)
    eta=(torch.exp(-gh/2)).float()
    etam1g=((-torch.expm1(-gh/2))/gam.double()).float()
    c11=(s/gam).float()
    c21=(torch.exp(-gh)*(torch.expm1(gh/2.0))**2/s).float()
    c22=(torch.sqrt(8*torch.expm1(-gh/2)-4*torch.expm1(-gh)-gh*torch.expm1(-gh))/s).float()
    hc=hclass(h=h,eta=eta,etam1g=etam1g,c11=c11,c21=c21,c22=c22)
    return(hc)

def BAOAB_hconst(h,gam):
    with torch.no_grad():
        hh=copy.deepcopy(h).detach().double()
        gamm=copy.deepcopy(gam).detach().double()
        gh=gamm*hh
        eta=(torch.exp(-gh/2))
        xc1=hh/2*(1+eta)
        xc2=(hh*hh/4)*(1+eta)
        xc3=hh/2*torch.sqrt(-torch.expm1(-gh))
        vc1=eta*(hh/2)
        vc2=(hh/2)
        vc3=torch.sqrt(-torch.expm1(-gh))

        hc=BAOABhclass(h=hh.float(),eta=eta.float(),xc1=xc1.float(),xc2=xc2.float(),xc3=xc3.float(),vc1=vc1.float(),vc2=vc2.float(),vc3=vc3.float())
        return(hc)

    # eta=to_data_type(exp(-h*gam/2));
    # h2=to_data_type(h/2);
    # eta2=to_data_type(exp(-h2*gam/2));

    # grad=gradp;
    # xip=R;

    # #xn=x+(h/2*(1+eta))*v-((h^2/4)*(1+eta))*grad+((h/2)*realsqrt(1-eta^2))*xip;
    # #=x+xc1*v-xc2*grad+xc3*xip

    # gradpn=grad_lpost(xn);
    # vn=eta*(v-(h/2)*grad)+(realsqrt(1-eta^2))*xip-(h/2)*gradpn;    
    # #vn=eta*v-vc1*grad-vc2*gradpn+vc3*xip

def U(x,v,hc,xi1,xi2):

    xn=x+hc.etam1g*v+hc.c11*xi1
    vn=v*hc.eta+hc.c21*xi1+hc.c22*xi2
    return([xn, vn])

def bounce(x,v,xstar,width):
    vsign=(((x-xstar+width)/(2*width)).floor()% 2)*(-2)+1
    vn=v*vsign
    xn=((x-xstar-width)% (4*width)-2*width).abs()-width+xstar  
    return([xn, vn])

def bouncenet():
    for p,p_star in zip(net.parameters(),net_star.parameters()):
        [p.data, p.v]=bounce(p.data, p.v, p_star.data, 6/torch.sqrt(l2regconst_extra))

def svrg_grad(net, batch_it):
    outputsU = net(images_list[batch_it])
    loss_likelihood = loss_function(outputsU, labels_list[batch_it])  


    grads_reg=[torch.zeros_like(par) for par in net.parameters()]
    net_pars=list(net.parameters())
    with torch.no_grad():
        for it in range(len_params):
            if(list_no_bias[it]):
                grads_reg[it]=net_pars[it].data*l2regconst

    net.zero_grad()
    loss_likelihood.backward()
    with torch.no_grad():
        grads_likelihood=[par.grad*batch_size for par in net.parameters()]

        svrg_grads=[]
        for p,grad,grad_reg,p_star,grad_star,star_sum_grad in zip(list(net.parameters()),grads_likelihood,grads_reg,list(net_star.parameters()),net_star_grad_list[batch_it],net_star_full_grad):              
            svrg_grads.append(grad_reg+star_sum_grad+(grad-grad_star)*no_batches+l2regconst_extra*(p.data-p_star.data))
    return svrg_grads,loss_likelihood.data


def UBU_step(hper2c,batch_it):   
    with torch.no_grad():
        for p in list(net.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper2c,xi1,xi2)


    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it)

    with torch.no_grad():

        for p,grad in zip(net.parameters(), svrg_grads):              
            #Using variance reduction
            p.v-=hper2c.h*grad
      
        for p in list(net.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper2c,xi1,xi2)

    return(loss_likelihood_data)



def UBU_step2(net, net2, hper4c,batch_it_list):   
    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)

    svrg_grads2,_=svrg_grad(net2, batch_it_list[0])
   
    with torch.no_grad():
        for q,gradq in zip(net2.parameters(), svrg_grads2):              
            q.v-=hper4c.h*gradq

    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)

    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it_list[2])

    with torch.no_grad():
        for p,grad in zip(net.parameters(), svrg_grads):              
            p.v-=2*hper4c.h*grad

    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)

    svrg_grads2,_=svrg_grad(net2, batch_it_list[1])

    with torch.no_grad():
        for q,grad in zip(net2.parameters(), svrg_grads2):              
            q.v-=hper4c.h*grad

    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)
    
    return(loss_likelihood_data)

def EM_step2(net, net2, h, gam, batch_it_list):   
    svrg_grads2,_=svrg_grad(net2, batch_it_list[0])
    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it_list[2])   
    sqrt2=torch.tensor(2).sqrt().detach()

    with torch.no_grad():
        for p,gradp in zip(net2.parameters(), svrg_grads2):              
            p.xi=torch.randn_like(p.data,device=device)
            p.data+=p.v*h/2
            p.v-=(h/2)*gradp+gam*(h/2)*p.v-torch.sqrt(2*gam*h/2)*p.xi

    svrg_grads2,_=svrg_grad(net2, batch_it_list[1])

    with torch.no_grad():
        for p,gradp in zip(net2.parameters(), svrg_grads2):              
            p.xi2=torch.randn_like(p.data,device=device)
            p.data+=p.v*h/2
            p.v-=(h/2)*gradp+gam*(h/2)*p.v-torch.sqrt(2*gam*h/2)*p.xi2

        for p,q,gradp in zip(net.parameters(),net2.parameters(), svrg_grads):              
            p.data+=p.v*h
            p.v-=(h)*gradp+gam*(h)*p.v-torch.sqrt(2*gam*h)*(q.xi+q.xi2)/sqrt2


    return(loss_likelihood_data)


def BAOAB_step(net,hc,batch_it,last_grad):   
    
    #xn=x+xc1*v-xc2*grad+xc3*xip
    with torch.no_grad():
        for p,grad in zip(net.parameters(), last_grad):
            p.xi=torch.randn_like(p.data,device=device)
            p.data=p.data+hc.xc1*p.v-hc.xc2*grad+hc.xc3*p.xi

    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it)

    with torch.no_grad():
        for p,grad,gradn in zip(net.parameters(), last_grad,svrg_grads):              
            p.v=hc.eta*p.v-hc.vc1*grad-hc.vc2*gradn+hc.vc3*p.xi

    return(loss_likelihood_data,svrg_grads)



def BAOAB_step2(net, net2, hc, hper2c, batch_it_list,last_grad,last_grad2):   

    with torch.no_grad():
        for p,grad in zip(net2.parameters(), last_grad2):
            p.xi=torch.randn_like(p.data,device=device)
            p.data=p.data+hper2c.xc1*p.v-hper2c.xc2*grad+hper2c.xc3*p.xi

    svrg_grads2,_=svrg_grad(net2, batch_it_list[0])

    with torch.no_grad():
        for p,grad,gradn in zip(net2.parameters(), last_grad2,svrg_grads2):              
            p.v=hper2c.eta*p.v-hper2c.vc1*grad-hper2c.vc2*gradn+hper2c.vc3*p.xi

    with torch.no_grad():
        for p,grad in zip(net2.parameters(), svrg_grads2):
            p.xi2=torch.randn_like(p.data,device=device)
            p.data=p.data+hper2c.xc1*p.v-hper2c.xc2*grad+hper2c.xc3*p.xi2

    svrg_grads2n,_=svrg_grad(net2, batch_it_list[1])

    with torch.no_grad():
        for p,grad,gradn in zip(net2.parameters(), svrg_grads2,svrg_grads2n):              
            p.v=hper2c.eta*p.v-hper2c.vc1*grad-hper2c.vc2*gradn+hper2c.vc3*p.xi2

    sqrt2=torch.tensor(2).sqrt().detach()
    with torch.no_grad():
        for p,q,grad in zip(net.parameters(),net2.parameters(), last_grad):
            p.data=p.data+hc.xc1*p.v-hc.xc2*grad+hc.xc3*(q.xi+q.xi2)/sqrt2

    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it_list[2])

    with torch.no_grad():
        for p,q,grad,gradn in zip(net.parameters(),net2.parameters(), last_grad,svrg_grads):              
            p.v=hc.eta*p.v-hc.vc1*grad-hc.vc2*gradn+hc.vc3*(q.xi+q.xi2)/sqrt2

    return(loss_likelihood_data,svrg_grads,svrg_grads2n)

    # eta=to_data_type(exp(-h*gam/2));
    # h2=to_data_type(h/2);
    # eta2=to_data_type(exp(-h2*gam/2));

    # grad=gradp;
    # xip=R;

    # #xn=x+(h/2*(1+eta))*v-((h^2/4)*(1+eta))*grad+((h/2)*realsqrt(1-eta^2))*xip;
    # #xn=x+xc1*v-xc2*grad+xc3*xip

    # gradpn=grad_lpost(xn);
    # vn=eta*(v-(h/2)*grad)+(realsqrt(1-eta^2))*xip-(h/2)*gradpn;    
    # #vn=eta*v-vc1*grad-vc2*gradpn+vc3*xip

def ind_create(batch_it):
    modit=batch_it %(2*no_batches)
    ind=(modit<=(no_batches-1))*modit+(modit>=no_batches)*(2*no_batches-modit-1)
    return ind


In [ ]:
#net = Fashion_MNIST_CNN().cuda()
#net2=copy.deepcopy(net)
#addnet(net2,net)
#multiplynet(net2,1/10)

In [ ]:
# filepath="output_fashion_low_rank_9n.pickle"
# #filepath="output_fashion_low_rank_long.pickle"
# with open(filepath,"rb") as file:
#    [labels_arr,test_labels_arr,test_prob_arr]=pickle.load(file)
# labels_arr=torch.tensor(labels_arr).detach()
# test_labels_arr=torch.tensor(test_labels_arr).detach()
# test_prob_arr=torch.tensor(test_prob_arr).detach()

In [ ]:


#@title Output arrays
num_classes=10

training_size=no_batches*batch_size
test_size=test_data_len


l2regconst=torch.tensor(50).detach()
l2regconst_extra=torch.tensor(0).detach()
gam=torch.sqrt(l2regconst)
#hper2c=hper2const(torch.tensor(h/2),gam)
all_images=torch.cat(images_list,dim=0).detach()
all_labels=torch.cat(labels_list,dim=0).detach()
# net = multinomial().cuda()
# net2 = multinomial().cuda()
# net.train()
# net2.train()

def find_MAP(num_steps):
  net = multinomial().cuda()

  def lpost():
    outputs = net(all_images)    
    loss_likelihood = loss_function(outputs, all_labels)
    reg=images_regulariser(net)
    loss=loss_likelihood+reg
    optimizer.zero_grad()
    loss.backward()
    return(loss)

  optimizer = torch.optim.LBFGS(net.parameters(), history_size=30, max_iter=20)
  for epoch in range(num_steps):
    optimizer.step(lpost)
  
  net_star=copy.deepcopy(net)
  len_params=len(list(net_star.parameters()))
  #Variance reduction - saving gradients at each batch at x_star
  net_star_grad_list=[]
  net_star_full_grad=[torch.zeros_like(par, device=device) for par in list(net_star.parameters())]
  for i in range(no_batches):
      images=images_list[i]
      labels=labels_list[i]
      outputs=net_star(images)
      loss_likelihood = loss_function(outputs, labels)
      reg=images_regulariser(net)
      net_star.zero_grad()
      loss_likelihood.backward()
      grads=[par.grad*batch_size for par in list(net_star.parameters())]
      net_star_grad_list.append(grads)
      for g, gi in zip(net_star_full_grad,grads):
        g+=gi          

  len_params=len(list(net_star.parameters()))
  list_no_bias=torch.zeros(len_params)
  pit=0
  for name, p in net_star.named_parameters():
      if 'bias' not in name:
          list_no_bias[pit]=1.0
      pit+=1
  return net_star, net_star_grad_list, net_star_full_grad, len_params, list_no_bias

  

def SMS_UBU2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)

  with torch.no_grad():
    hper4c=hper2const(torch.tensor(h/2),gam)
    test_labels_arr=torch.zeros(test_size).detach()
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs]).detach()
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs]).detach()  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()

  for epoch in range(num_epochs):
    sum_loss=0
    if(epoch%2==0):
      rperm=random.permutation(list(range(no_batches)))      
    rperm2=random.permutation(list(range(no_batches)))
    for i in range(no_batches):
      b=i        
      it=epoch*batch_size+b
      ind=ind_create(2*it)
      ind2=ind_create(2*it+1)            
      indc=ind_create(it)          
      batch_it_list=[rperm2[ind], rperm2[ind2], rperm[indc]]

      loss_likelihood=UBU_step2(net,net2,hper4c,batch_it_list)
      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2= net2(imagest).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)

        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)
  
def SG_UBU2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)
  with torch.no_grad():
    hper4c=hper2const(torch.tensor(h/2),gam)
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()


  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):
      ind=torch.randint(high=no_batches,size=(1,1))
      ind2=torch.randint(high=no_batches,size=(1,1))
      if(torch.rand(1)<0.5):
        indc=ind
      else:
        indc=ind2
      batch_it_list=[ind, ind2, indc]

      loss_likelihood=UBU_step2(net,net2,hper4c,batch_it_list)
      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2= net2(imagest).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)

        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)

def SG_EM2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)
  with torch.no_grad():
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()

  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):
      ind=torch.randint(high=no_batches,size=(1,1))
      ind2=torch.randint(high=no_batches,size=(1,1))
      if(torch.rand(1)<0.5):
        indc=ind
      else:
        indc=ind2
      batch_it_list=[ind, ind2, indc]

      loss_likelihood=EM_step2(net,net2,h,gam,batch_it_list)
      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2= net2(imagest).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)

        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)

def SG_BAOAB(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  
  with torch.nograd():
    hc=BAOAB_hconst(h,gam)
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    #Initialise velocities
    for par in list(net.parameters()):
      par.v = torch.randn_like(par,device=device)      

  
  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):
      ind=torch.randint(high=no_batches,size=(1,1))
      if(epoch==0 and i==0):
        last_grad,_=svrg_grad(net,ind)
      loss_likelihood,last_grad=BAOAB_step(net,hc,ind,last_grad)

      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr)



def SG_BAOAB2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)  
  with torch.no_grad():
    hc=BAOAB_hconst(h,gam)
    hper2c=BAOAB_hconst(h/2,gam)      
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()
  
  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):

      ind=torch.randint(high=no_batches,size=(1,1))
      ind2=torch.randint(high=no_batches,size=(1,1))
      if(torch.rand(1)<0.5):
        indc=ind
      else:
        indc=ind2
      batch_it_list=[ind, ind2, indc]

      if(epoch==0 and i==0):
        last_grad,_=svrg_grad(net,ind)
        last_grad2,_=svrg_grad(net2,indc)

      loss_likelihood,last_grad,last_grad2=BAOAB_step2(net,net2,hc,hper2c,batch_it_list,last_grad,last_grad2)

      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2 = net2(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)
        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)


def SMS_BAOAB2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)  
  with torch.no_grad():
    hc=BAOAB_hconst(h,gam)
    hper2c=BAOAB_hconst(h/2,gam)      
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()
  
  for epoch in range(num_epochs):
    sum_loss=0
    if(epoch%2==0):
      rperm=random.permutation(list(range(no_batches)))      
    rperm2=random.permutation(list(range(no_batches)))
    for i in range(no_batches):
      b=i        
      it=epoch*batch_size+b
      ind=ind_create(2*it)
      ind2=ind_create(2*it+1)            
      indc=ind_create(it)          
      batch_it_list=[rperm2[ind], rperm2[ind2], rperm[indc]]

      if(epoch==0 and i==0):
        last_grad,_=svrg_grad(net,ind)
        last_grad2,_=svrg_grad(net2,indc)

      loss_likelihood,last_grad,last_grad2=BAOAB_step2(net,net2,hc,hper2c,batch_it_list,last_grad,last_grad2)

      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2 = net2(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)
        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)

net_star, net_star_grad_list, net_star_full_grad, len_params, list_no_bias=find_MAP(40)


In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())


In [ ]:
rat=2
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=800*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h4l.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,10*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,10*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=64
num_epochs=800
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
# filepath="output_fashion_multinomial_SG_EM_h5.pickle"
# with open(filepath,"wb") as file:
#   pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
len_params=len(list(net_star.parameters()))
list_no_bias=torch.zeros(len_params)
it=0
for name, p in net_star.named_parameters():
    if 'bias' not in name:
        list_no_bias[it]=1.0
    it+=1

def hess_vec(vec):   
    out=net_star(images)
    loss=loss_function(out,labels)*train_data_len
    net_star.zero_grad()
    grad=list(torch.autograd.grad(loss,list(net_star.parameters()),create_graph=True))
    res=torch.zeros(1).cuda()
    for it in range(len_params):
        res+=(grad[it]*vec[it]).sum()

    net_star.zero_grad()
    hvp=list(torch.autograd.grad(res,list(net_star.parameters()),create_graph=False))

    with torch.no_grad():
        for it in range(len_params):
            if(list_no_bias[it]):
                hvp[it]+=vec[it]*l2regconst
    return(hvp)

def hess_vec_full(vec):
    hvfull=copy.deepcopy(list(vec))
    for p in hvfull:
        p.requires_grad=False
        p*=0
    len_pars=len(hvfull)
    # for it in range(no_batches):
    #     images=images_list[it]
    #     labels=labels_list[it]
    #     hv=hess_vec(vec)
    #     for pit in range(len_pars):
    #         hvfull[pit]+=hv[pit]
    images=all_images
    labels=all_labels
    hv=hess_vec(vec)
    for pit in range(len_pars):
        hvfull[pit]+=hv[pit]

    return(hvfull)

def norm_par(vec):
    n=torch.zeros(1).detach().cuda()
    for p in vec:
        n+=p.pow(2).sum()
    return n.sqrt()

def multiply_par(vec,c):
    res_vec=copy.deepcopy(vec)
    for it in range(len_params):
        res_vec[it].requires_grad=False
        res_vec[it]=res_vec[it]*c
    return res_vec

def scalar_prod_par(vec1,vec2):
    res=torch.zeros(1).cuda()
    for it in range(len_params):
        res+=(vec1[it]*vec2[it]).sum()
    return res

def add_par(vec1,vec2):
    res_vec=copy.deepcopy(vec1)
    for it in range(len_params):
        res_vec[it].requires_grad=False
        res_vec[it]=vec1[it]+vec2[it]
    return res_vec

# print(norm_par(list(net_star.parameters())))
# print(norm_par(multiply_par(list(net_star.parameters()),0.1)))

vec=copy.deepcopy(list(net_star.parameters()))
for p in vec:
    p.requires_grad=False
norm_iter=norm_par(vec)
vec=multiply_par(vec,torch.Tensor(1/norm_iter))
for iter in range(20):
    vec=hess_vec_full(vec)
    norm_iter=norm_par(vec)
    vec=multiply_par(vec,1/norm_iter)    
    print(norm_iter)

max_eigen_vec=copy.deepcopy(vec)
max_eigen=norm_iter

vec=copy.deepcopy(list(net_star.parameters()))
for p in vec:
    p.requires_grad=False
vec=add_par(vec,multiply_par(max_eigen_vec,-scalar_prod_par(vec,max_eigen_vec)))
norm_iter=norm_par(vec)
vec=multiply_par(vec,torch.Tensor(1/norm_iter))

for iter in range(20):
    vec=hess_vec_full(vec)
    vec=add_par(vec,multiply_par(max_eigen_vec,-scalar_prod_par(vec,max_eigen_vec)))
    norm_iter=norm_par(vec)
    vec=multiply_par(vec,1/norm_iter)    
    print(norm_iter)


# hvpf=hess_vec_full(vtest)
# nhvp=0
# for p in hvpf:
#     nhvp+=p.abs().sum()
# print(nhvp)

In [ ]:
#filepath="output_fashion.pickle"
#with open(filepath,"rb") as file:
#    [labels_arr,test_labels_arr,test_prob_arr,_,_]=pickle.load(file)
#labels_arr=torch.tensor(labels_arr).detach()
#test_labels_arr=torch.tensor(test_labels_arr).detach()
#test_prob_arr=torch.tensor(test_prob_arr).detach()

In [ ]:
# Outcome should be a binary list of the ordinal outcome. [0, 1, 0] for exmaple.
# Probs should be a list of probabilities. [0.79, 0.09, 0.12] for example.
# Outcome and Probs must be provided with the same order as probabilities.

def rps_single(probs, true_label):
    outcome=torch.zeros(num_classes)
    outcome[true_label.int()]=1.0
    cum_probs = torch.cumsum(probs,0)
    cum_outcomes = torch.cumsum(outcome,0)
    
    #print(cum_outcomes)
    #print(cum_probs)
    sum_rps = 0
    for i in range(len(outcome)):         
        sum_rps+= (cum_probs[i] - cum_outcomes[i])**2
    
    return sum_rps/(num_classes-1)

def rps_calc(test_probs, true_labels):
    rps_vec=torch.zeros(test_data_len)
    for it in range(test_data_len):
        rps_vec[it]=rps_single(test_probs[it,:].reshape(num_classes),true_labels[it])
    return rps_vec

def nll_calc(test_probs, true_labels):
    res=0
    for it in range(test_data_len):
        res-=torch.log(test_probs[it,true_labels[it].int()])
    return res/test_data_len

def adaptive_calibration_error(test_probs,true_labels, num_bins=20):
    max_probs, predicted_labels = torch.max(test_probs,1)
    ind=torch.argsort(max_probs,stable=True)
    sorted_max_probs=max_probs[ind]
    sorted_predicted_labels=predicted_labels[ind]
    sorted_true_labels=true_labels[ind]

    correct = (sorted_predicted_labels == sorted_true_labels).clone().detach().float()
    bins=(torch.tensor(range(test_data_len))/torch.tensor(test_data_len/num_bins)).floor()

    o=torch.tensor(0.0)
    for b in range(num_bins):
        mask = (bins == b)
        if torch.any(mask):
            #print(b, sorted_max_probs[mask].mean(), (correct[mask] - sorted_max_probs[mask]).mean())
            o += (correct[mask] - sorted_max_probs[mask]).mean().abs()

    return o / num_bins

In [ ]:
# def GRdiagnostics(res):
#   J=res.shape[0] #Number of chains
#   L=res.shape[1] #Number of samples after burnin
#   res_means=res.mean(dim=1)
#   res_mean=res_means.mean()
#   B=(res_means-res_mean).pow(2).sum()*L/(J-1)
#   W=(res_means.reshape([J,1])@torch.ones([1,L])-res).pow(2).sum()/(J*(L-1))
#   R=(W*(L-1)/L+B/L)/W
#   return R


# par_chains=4
# no_GR_epochs=40
# test_prob_GR_arr=torch.zeros([test_size,num_classes])
# nll_GR_arr=torch.zeros([par_chains,no_GR_epochs])
# for chain in range(par_chains):
#     net=copy.deepcopy(net_star)
#     net.eval()
#     for par in list(net.parameters()):
#       par.v = torch.randn_like(par,device=device)          
#     for epoch in range(no_GR_epochs):
#       print("chain: ",chain, "/epoch:",epoch)
#       if(epoch % 2 == 1):
#         irange=range(no_batches-1,-1,-1)
#       else:
#         irange=range(no_batches)
#       for b in irange:
#         images=images_list[b]
#         labels=labels_list[b]
#         UBU_step(hper2c,images,labels,b)

#       for testit in range(test_no_batches):
#         imagest=test_images_list[testit]
#         labelst=test_labels_list[testit]
#         actual_batch_size=len(imagest)
#         outputt = net(imagest).detach()
#         test_prob_GR_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),:]=torch.softmax(outputt,dim=1)
      
#       nll_GR_arr[chain,epoch]=nll_calc(test_prob_GR_arr,test_labels_arr)
#       print("NLL:", nll_GR_arr[chain,epoch])

# print(GRdiagnostics(nll_GR_arr))

In [ ]:
#test_prob=torch.Tensor(test_prob_arr[:,:,29,0]).reshape(test_size,num_classes)
#torch.cumsum(test_prob[1,:].reshape(num_classes),0)
#rps_single(test_prob[1,:].reshape(num_classes),test_labels_arr[1])


In [ ]:
num_runs=1
#no bayesian
def compute_acc_ace_rps_no_bayes(es):
    copies=int(num_runs/es)
    ace_arr=torch.zeros(copies)
    rps_arr=torch.zeros(copies)
    nll_arr=torch.zeros(copies)
    accuracy_arr=torch.zeros(copies)

    for it in range(copies):
        test_prob=torch.Tensor(test_prob_arr[:,:,14,it*es:(it+1)*es]).mean(-1).reshape(test_size,num_classes)
        ace_arr[it]=adaptive_calibration_error(test_prob,test_labels_arr)
        rps_arr[it]=(rps_calc(test_prob, test_labels_arr)).mean()
        nll_arr[it]=nll_calc(test_prob, test_labels_arr)
        _, predictedt = torch.max(test_prob,1)
        accuracy_arr[it]= (predictedt==test_labels_arr.reshape(1,test_size)).sum()/test_size
    print("Non-Bayesian, ensemble size:", es)
    print("mean accuracy:",accuracy_arr.mean(),"std:",accuracy_arr.std())
    print("mean ace:",ace_arr.mean(),"std:",ace_arr.std())
    print("mean nll:",nll_arr.mean(),"std:",nll_arr.std())
    print("mean rps:",rps_arr.mean(),"std:",rps_arr.std())
    return [accuracy_arr.mean(),accuracy_arr.std(),ace_arr.mean(),ace_arr.std(),rps_arr.mean(),rps_arr.std(),nll_arr.mean(),nll_arr.std()]

acc=torch.zeros(5)
acc_std=torch.zeros(5)
ace=torch.zeros(5)
ace_std=torch.zeros(5)
rps=torch.zeros(5)
rps_std=torch.zeros(5)
nll=torch.zeros(5)
nll_std=torch.zeros(5)
[acc[0],acc_std[0],ace[0],ace_std[0],rps[0],rps_std[0],nll[0],nll_std[0]]=compute_acc_ace_rps_no_bayes(1)
[acc[1],acc_std[1],ace[1],ace_std[1],rps[1],rps_std[1],nll[1],nll_std[1]]=compute_acc_ace_rps_no_bayes(2)
[acc[2],acc_std[2],ace[2],ace_std[2],rps[2],rps_std[2],nll[2],nll_std[2]]=compute_acc_ace_rps_no_bayes(4)
[acc[3],acc_std[3],ace[3],ace_std[3],rps[3],rps_std[3],nll[3],nll_std[3]]=compute_acc_ace_rps_no_bayes(8)
[acc[4],acc_std[4],ace[4],ace_std[4],rps[4],rps_std[4],nll[4],nll_std[4]]=compute_acc_ace_rps_no_bayes(16)

# from scipy.io import savemat
# filepath="results_fashion_no_bayes.mat"
# mdic={"acc":acc.cpu().numpy(),"acc_std":acc_std.cpu().numpy(),"nll": nll.cpu().numpy(),"nll_std":nll_std.cpu().numpy(),\
#       "ace": ace.cpu().numpy(),"ace_std":ace_std.cpu().numpy(), "rps":rps.cpu().numpy(),"rps_std":rps_std.cpu().numpy()}
# savemat(filepath,mdic)

Non-Bayesian, ensemble size: 1
mean accuracy: tensor(0.9296) std: tensor(0.0020)
mean ace: tensor(0.0555) std: tensor(0.0018)
mean nll: tensor(0.4709) std: tensor(0.0251)
mean rps: tensor(0.0230) std: tensor(0.0008)
Non-Bayesian, ensemble size: 2
mean accuracy: tensor(0.9383) std: tensor(0.0015)
mean ace: tensor(0.0294) std: tensor(0.0016)
mean nll: tensor(0.3109) std: tensor(0.0127)
mean rps: tensor(0.0188) std: tensor(0.0005)
Non-Bayesian, ensemble size: 4
mean accuracy: tensor(0.9422) std: tensor(0.0010)
mean ace: tensor(0.0177) std: tensor(0.0011)
mean nll: tensor(0.2385) std: tensor(0.0066)
mean rps: tensor(0.0168) std: tensor(0.0003)
Non-Bayesian, ensemble size: 8
mean accuracy: tensor(0.9443) std: tensor(0.0006)
mean ace: tensor(0.0126) std: tensor(0.0007)
mean nll: tensor(0.2025) std: tensor(0.0042)
mean rps: tensor(0.0157) std: tensor(0.0002)
Non-Bayesian, ensemble size: 16
mean accuracy: tensor(0.9459) std: tensor(0.0009)
mean ace: tensor(0.0098) std: tensor(0.0007)
mean nll: tensor(0.1832) std: tensor(0.0029)
mean rps: tensor(0.0152) std: tensor(8.2395e-05)

In [ ]:
#swa
def compute_acc_ace_rps_swa(es):
    copies=int(num_runs/es)
    ace_arr=torch.zeros(copies)
    rps_arr=torch.zeros(copies)
    nll_arr=torch.zeros(copies)
    accuracy_arr=torch.zeros(copies)

    for it in range(copies):
        test_prob=torch.Tensor(test_prob_arr[:,:,19,it*es:(it+1)*es]).mean(-1).reshape(test_size,num_classes)
        ace_arr[it]=adaptive_calibration_error(test_prob,test_labels_arr)
        rps_arr[it]=(rps_calc(test_prob, test_labels_arr)).mean()
        nll_arr[it]=nll_calc(test_prob, test_labels_arr)
        _, predictedt = torch.max(test_prob,1)
        accuracy_arr[it]= (predictedt==test_labels_arr.reshape(1,test_size)).sum()/test_size
    print("SWA, ensemble size:", es)
    print("mean accuracy:",accuracy_arr.mean(),"std:",accuracy_arr.std())
    print("mean ace:",ace_arr.mean(),"std:",ace_arr.std())
    print("mean nll:",nll_arr.mean(),"std:",nll_arr.std())
    print("mean rps:",rps_arr.mean(),"std:",rps_arr.std())
    return [accuracy_arr.mean(),accuracy_arr.std(),ace_arr.mean(),ace_arr.std(),rps_arr.mean(),rps_arr.std(),nll_arr.mean(),nll_arr.std()]

acc=torch.zeros(5)
acc_std=torch.zeros(5)
ace=torch.zeros(5)
ace_std=torch.zeros(5)
rps=torch.zeros(5)
rps_std=torch.zeros(5)
nll=torch.zeros(5)
nll_std=torch.zeros(5)
[acc[0],acc_std[0],ace[0],ace_std[0],rps[0],rps_std[0],nll[0],nll_std[0]]=compute_acc_ace_rps_swa(1)
[acc[1],acc_std[1],ace[1],ace_std[1],rps[1],rps_std[1],nll[1],nll_std[1]]=compute_acc_ace_rps_swa(2)
[acc[2],acc_std[2],ace[2],ace_std[2],rps[2],rps_std[2],nll[2],nll_std[2]]=compute_acc_ace_rps_swa(4)
[acc[3],acc_std[3],ace[3],ace_std[3],rps[3],rps_std[3],nll[3],nll_std[3]]=compute_acc_ace_rps_swa(8)
[acc[4],acc_std[4],ace[4],ace_std[4],rps[4],rps_std[4],nll[4],nll_std[4]]=compute_acc_ace_rps_swa(16)

# from scipy.io import savemat
# filepath="results_fashion_swa.mat"
# mdic={"acc":acc.cpu().numpy(),"acc_std":acc_std.cpu().numpy(),"nll": nll.cpu().numpy(),"nll_std":nll_std.cpu().numpy(),\
#       "ace": ace.cpu().numpy(),"ace_std":ace_std.cpu().numpy(), "rps":rps.cpu().numpy(),"rps_std":rps_std.cpu().numpy()}
# savemat(filepath,mdic)

SWA, ensemble size: 1
mean accuracy: tensor(0.9337) std: tensor(0.0016)
mean ace: tensor(0.0537) std: tensor(0.0016)
mean nll: tensor(0.4826) std: tensor(0.0191)
mean rps: tensor(0.0220) std: tensor(0.0006)
SWA, ensemble size: 2
mean accuracy: tensor(0.9403) std: tensor(0.0012)
mean ace: tensor(0.0309) std: tensor(0.0012)
mean nll: tensor(0.3312) std: tensor(0.0111)
mean rps: tensor(0.0184) std: tensor(0.0004)
SWA, ensemble size: 4
mean accuracy: tensor(0.9435) std: tensor(0.0011)
mean ace: tensor(0.0223) std: tensor(0.0010)
mean nll: tensor(0.2579) std: tensor(0.0078)
mean rps: tensor(0.0166) std: tensor(0.0003)
SWA, ensemble size: 8
mean accuracy: tensor(0.9453) std: tensor(0.0008)
mean ace: tensor(0.0180) std: tensor(0.0011)
mean nll: tensor(0.2186) std: tensor(0.0036)
mean rps: tensor(0.0157) std: tensor(0.0002)
SWA, ensemble size: 16
mean accuracy: tensor(0.9462) std: tensor(0.0008)
mean ace: tensor(0.0158) std: tensor(0.0009)
mean nll: tensor(0.1964) std: tensor(0.0034)
mean rps: tensor(0.0153) std: tensor(5.5335e-05)

In [ ]:
#Bayesian
def compute_acc_ace_rps_bayes(es):
    copies=int(num_runs/es)
    ace_arr=torch.zeros(copies)
    rps_arr=torch.zeros(copies)
    nll_arr=torch.zeros(copies)
    accuracy_arr=torch.zeros(copies)

    for it in range(copies):
        test_prob=torch.Tensor(test_prob_arr[:,:,30:60,it*es:(it+1)*es]).mean(-1).mean(-1).reshape(test_size,num_classes)
        ace_arr[it]=adaptive_calibration_error(test_prob,test_labels_arr)
        rps_arr[it]=(rps_calc(test_prob, test_labels_arr)).mean()
        nll_arr[it]=nll_calc(test_prob, test_labels_arr)
        _, predictedt = torch.max(test_prob,1)
        accuracy_arr[it]= (predictedt==test_labels_arr.reshape(1,test_size)).sum()/test_size
    print("Bayesian, ensemble size:", es)
    print("mean accuracy:",accuracy_arr.mean(),"std:",accuracy_arr.std())
    print("mean ace:",ace_arr.mean(),"std:",ace_arr.std())
    print("mean nll:",nll_arr.mean(),"std:",nll_arr.std())
    print("mean rps:",rps_arr.mean(),"std:",rps_arr.std())
    return [accuracy_arr.mean(),accuracy_arr.std(),ace_arr.mean(),ace_arr.std(),rps_arr.mean(),rps_arr.std(),nll_arr.mean(),nll_arr.std()]



acc=torch.zeros(5)
acc_std=torch.zeros(5)
ace=torch.zeros(5)
ace_std=torch.zeros(5)
rps=torch.zeros(5)
rps_std=torch.zeros(5)
nll=torch.zeros(5)
nll_std=torch.zeros(5)
[acc[0],acc_std[0],ace[0],ace_std[0],rps[0],rps_std[0],nll[0],nll_std[0]]=compute_acc_ace_rps_bayes(1)
[acc[1],acc_std[1],ace[1],ace_std[1],rps[1],rps_std[1],nll[1],nll_std[1]]=compute_acc_ace_rps_bayes(2)
[acc[2],acc_std[2],ace[2],ace_std[2],rps[2],rps_std[2],nll[2],nll_std[2]]=compute_acc_ace_rps_bayes(4)
[acc[3],acc_std[3],ace[3],ace_std[3],rps[3],rps_std[3],nll[3],nll_std[3]]=compute_acc_ace_rps_bayes(8)
[acc[4],acc_std[4],ace[4],ace_std[4],rps[4],rps_std[4],nll[4],nll_std[4]]=compute_acc_ace_rps_bayes(16)

# from scipy.io import savemat
# filepath="results_fashion_bayes.mat"
# mdic={"acc":acc.cpu().numpy(),"acc_std":acc_std.cpu().numpy(),"nll": nll.cpu().numpy(),"nll_std":nll_std.cpu().numpy(),\
#       "ace": ace.cpu().numpy(),"ace_std":ace_std.cpu().numpy(), "rps":rps.cpu().numpy(),"rps_std":rps_std.cpu().numpy()}
# savemat(filepath,mdic)

Bayesian, ensemble size: 1
mean accuracy: tensor(0.9358) std: tensor(nan)
mean ace: tensor(0.0123) std: tensor(nan)
mean nll: tensor(0.2039) std: tensor(nan)
mean rps: tensor(0.0177) std: tensor(nan)

Bayesian, ensemble size: 1
mean accuracy: tensor(0.9400) std: tensor(nan)
mean ace: tensor(0.0072) std: tensor(nan)
mean nll: tensor(0.1876) std: tensor(nan)
mean rps: tensor(0.0169) std: tensor(nan)


In [ ]:
# test_prob=torch.Tensor(test_prob_arr[:,7,0]).reshape(test_size,1)
# ace=adaptive_calibration_error(test_labels_arr.reshape(test_size,1).numpy(),test_prob.numpy(),20)
# ace

In [ ]:
# test_prob=torch.Tensor(test_prob_arr[:,10:16,48:64]).mean(-1).mean(-1).reshape(test_size,1)
# ace=adaptive_calibration_error(test_labels_arr.reshape(test_size,1).numpy(),test_prob.numpy(),20)
# ace

In [ ]:
import torchvision
from torchview import draw_graph
net2 = Fashion_MNIST_CNN()

#model_graph = draw_graph(net2, input_size=(1,3,64,64), expand_nested=True)
model_graph = draw_graph(net2, input_size=(1,1,28,28), expand_nested=True)
model_graph.visual_graph
